In [1]:
#####################################################
# ET-robot-contest Game Area Detection and find block color.
# Copyright © 2022 naoki hunada. All rights reserved.
#####################################################

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

import sys

import area_detect
import perspective
import block_detect

In [3]:
param1 = area_detect.option()
param2 = block_detect.option()
ad = area_detect.AreaDetection()
bd = block_detect.blockAreaDetect()

In [4]:
def init():
    #エリア検知パラメータ
    param1.PROCESS_IMAGE_SIZE = [300, 320]
    param1.IMAGE_MASK_MIN_THRESHOLD = [0,51,70] #HSV
    param1.IMAGE_MASK_MAX_THRESHOLD = [255,255,255] #HSV
    param1.LINE_COLOR_DETECTION_FINENESS = 30
    param1.LINE_COLOR_DETECTION_SCORE_THRESHOLD = 550
    param1.LINE_COLOR_DETECTION_MAX_THRESHOLD = [255,255,45] #HSV
    param1.LINE_DETECTION_ROTATE_RANGE = 3
    param1.POINT_DETECTION_MIN_AREA_THRESHOLD = 30
    param1.POINT_DETECTION_MAX_AREA_THRESHOLD = 450
    param1.BLUE_CIRCLE_DETECTION_MIN_THRESHOLD = [50, 0, 0] #HSV
    param1.BLUE_CIRCLE_DETECTION_MAX_THRESHOLD = [90, 255, 255] #HSV
    param1.GREEN_CIRCLE_DETECTION_MIN_THRESHOLD = [90, 0, 0] #HSV
    param1.GREEN_CIRCLE_DETECTION_MAX_THRESHOLD = [125, 255, 255] #HSV
    ad.set_option(param1)

    #ブロック検知パラメータ
    param2.BLOCK_POS = [[150, 12], [288, 150], [150, 288], [12, 150],
        [-1, -1],
        [86, 86], [150, 86], [216, 86],
        [86, 150], [216, 150],
        [86, 216], [150, 216], [216, 216]]
    param2.RED_BLOCK_MIN_THRESHOLD1 = [0, 40, 10] #HSV
    param2.RED_BLOCK_MAX_THRESHOLD1 = [20, 255, 255] #HSV
    param2.RED_BLOCK_MIN_THRESHOLD2 = [160, 40, 10] #HSV
    param2.RED_BLOCK_MAX_THRESHOLD2 = [179, 255, 255] #HSV
    param2.GREEN_BLOCK_MIN_THRESHOLD = [45, 40, 10] #HSV
    param2.GREEN_BLOCK_MAX_THRESHOLD = [80, 255, 255] #HSV
    param2.BLUE_BLOCK_MIN_THRESHOLD = [105, 40, 10] #HSV
    param2.BLUE_BLOCK_MAX_THRESHOLD = [140, 255, 255] #HSV
    param2.YELLOW_BLOCK_MIN_THRESHOLD = [20, 40, 10] #HSV
    param2.YELLOW_BLOCK_MAX_THRESHOLD = [45, 255, 255] #HSV
    bd.set_option(param2)

init()

In [5]:
### カメラデバイスを使う(リアルタイム)
def camera():
    window_name = "camera_et"
    cap = cv2.VideoCapture(1)

    if not cap.isOpened():
        sys.exit()
    ret, frame = cap.read()
    width = frame.shape[0]
    height = frame.shape[1]
    s_point = -1

    while True:
        #カメラからの画像取得
        ret, frame = cap.read()
        if not ret:
            break
        out_img, point, success = ad.detect(frame)
        img = out_img[0]
        bonus_img = img
        if(success == True):
            point = [point[0], point[1], point[2], point[3]]
            s_point = point
        if(not(s_point == -1)):
            img, bonus_img = perspective.area_perspective_transform(img, s_point, 82)
        color_block, success = bd.detect_block(img)
        color_block[4], succes = bd.detect_bonus_block(bonus_img)
        print(color_block)
        #img = cv2.resize(img, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))) #(int(width*2), int(height*2))
        #カメラの画像の出力
        cv2.imshow('camera' , img)

        #繰り返し分から抜けるためのif文
        key = cv2.waitKey(33)
        if key == 27:
            break

    #メモリを解放して終了するためのコマンド
    cap.release()
    cv2.destroyAllWindows()

In [6]:
### 動画データを使う


def video(file_name):
    window_name = "video_et"
    cap_file = cv2.VideoCapture(file_name)

    if not cap_file.isOpened():
        sys.exit()
    ret, frame = cap_file.read()
    width = frame.shape[0]
    height = frame.shape[1]
    s_point = -1

    wait_time = int(round(cap_file.get(cv2.CAP_PROP_FPS), 0))
    while True:
        ret, frame = cap_file.read()
        if not ret:
            break
        out_img, point, success = ad.detect(frame)
        img = out_img[2]
        bonus_img = img
        if(success == True):
            point = [point[0], point[1], point[2], point[3]]
            s_point = point
        if(not(s_point == -1)):
            img, bonus_img = perspective.area_perspective_transform(img, s_point, 82)
        color_block, success = bd.detect_block(img)
        color_block[4], success = bd.detect_bonus_block(bonus_img)
        print(color_block)
        #img = cv2.resize(img, (int(cap_file.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap_file.get(cv2.CAP_PROP_FRAME_HEIGHT)))) #(int(width*1.2), int(height*1.2))
        cv2.imshow(window_name, img)

        key = cv2.waitKey(wait_time)
        if key == 27:
            break

    cap_file.release()
    cv2.destroyWindow(window_name)

In [7]:
###画像データを使う
def image(file_name):
    img = cv2.imread(file_name)
    bonus_img = img
    out_img, point, success = ad.detect(img)
    img = out_img[0]
    if(success == True):
        point = [point[0], point[1], point[2], point[3]]
        img, bonus_img = perspective.area_perspective_transform(img, point, 82)
    color_block, success = bd.detect_block(img)
    bonus_block, success = bd.detect_bonus_block(bonus_img)
    print(color_block)
    cv2.imwrite("output.png",np.asarray(img))
    plt.imshow(np.asarray(img))

In [8]:
##### execute #####

#image("test.png")
video("video4.mp4")
#camera()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
